In [2]:
##Only run if you need the news headlines! This will download the headlines from the internet into a new csv every time!!!

!wget https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv

--2024-05-10 20:41:32--  https://bitbucket.org/newsstuff/headlinesdata/raw/3d468c6c414ffb591f6e685644da25aab1621bdf/stock_news.csv
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2227283 (2.1M) [text/plain]
Saving to: ‘stock_news.csv.1’

stock_news.csv.1    100%[===================>]   2.12M  2.96MB/s    in 0.7s    

2024-05-10 20:41:33 (2.96 MB/s) - ‘stock_news.csv.1’ saved [2227283/2227283]



In [3]:
import pandas as pd

df = pd.read_csv('stock_news.csv')


In [4]:
with open('../headlinesData/Good.csv') as good:
    good = good.read().split('\n')
    
with open('../headlinesData/Bad.csv') as bad:
    bad = bad.read().split('\n')
    
with open('../headlinesData/Neutral.csv') as neutral:
    neutral = neutral.read().split('\n')

In [5]:
del df['Unnamed: 0']

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import vader

In [7]:
v_t = TfidfVectorizer()
bad_vc = CountVectorizer()
good_vc = CountVectorizer()
neutral_vc = CountVectorizer()
sia = vader.SentimentIntensityAnalyzer()

In [8]:
df['headline'] = df['headline'].apply(lambda x: x.lower())  

df_good = df[df['label'] == 'Positive']
df_bad = df[df['label'] == 'Negative']
df_neutral = df[df['label'] == 'Neutral']

In [9]:
print(type(df_good))
print(type(df_bad))
print(type(df_neutral))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [10]:
# Convert 'good', 'bad', and 'neutral' to dataframes
df_good_new = pd.DataFrame(good, columns=['headline'])
df_bad_new = pd.DataFrame(bad, columns=['headline'])
df_neutral_new = pd.DataFrame(neutral, columns=['headline'])

# Combine the 'good', 'bad', and 'neutral' data with their respective dataframes
# Combine the 'good', 'bad', and 'neutral' data with their respective dataframes
df_good = pd.concat([df_good, df_good_new], ignore_index=True)
df_bad = pd.concat([df_bad, df_bad_new], ignore_index=True)
df_neutral = pd.concat([df_neutral, df_neutral_new], ignore_index=True)

# Shuffle the dataframes
df_good = df_good.sample(frac=1).reset_index(drop=True)
df_bad = df_bad.sample(frac=1).reset_index(drop=True)
df_neutral = df_neutral.sample(frac=1).reset_index(drop=True)

# Fit the vectorizers to the updated dataframes
v_t.fit(df['headline'])

good_vc.fit(df_good['headline'])
bad_vc.fit(df_bad['headline'])
neutral_vc.fit(df_neutral['headline'])

CountVectorizer()

In [11]:
v_t.fit(df['headline'])

good_vc.fit(df_good['headline'])
bad_vc.fit(df_bad['headline'])
neutral_vc.fit(df_neutral['headline'])

CountVectorizer()

In [12]:
df_good_sia = df_good['headline'].apply(lambda x: sia.polarity_scores(x))
df_bad_sia = df_bad['headline'].apply(lambda x: sia.polarity_scores(x))
df_neutral_sia = df_neutral['headline'].apply(lambda x: sia.polarity_scores(x))

In [13]:
df_good_bad_vc = bad_vc.transform(df_good['headline'])
df_bad_bad_vc = bad_vc.transform(df_bad['headline'])
df_neutral_bad_vc = bad_vc.transform(df_neutral['headline'])

df_good_good_vc = good_vc.transform(df_good['headline'])
df_bad_good_vc = good_vc.transform(df_bad['headline'])
df_neutral_good_vc = good_vc.transform(df_neutral['headline'])

df_good_neutral_vc = neutral_vc.transform(df_good['headline'])
df_bad_neutral_vc = neutral_vc.transform(df_bad['headline'])
df_neutral_neutral_vc = neutral_vc.transform(df_neutral['headline'])

df_good_v_t = v_t.transform(df_good['headline'])
df_bad_v_t = v_t.transform(df_bad['headline'])
df_neutral_v_t = v_t.transform(df_neutral['headline'])

In [14]:
good_features = np.hstack([df_good_sia.apply(pd.Series).values, df_good_bad_vc.toarray(), df_good_good_vc.toarray(), df_good_neutral_vc.toarray(), df_good_v_t.toarray()])
bad_features = np.hstack([df_bad_sia.apply(pd.Series).values, df_bad_bad_vc.toarray(), df_bad_good_vc.toarray(), df_bad_neutral_vc.toarray(), df_bad_v_t.toarray()])
neutral_features = np.hstack([df_neutral_sia.apply(pd.Series).values, df_neutral_bad_vc.toarray(), df_neutral_good_vc.toarray(), df_neutral_neutral_vc.toarray(), df_neutral_v_t.toarray()])

In [15]:
good_labels = np.ones(good_features.shape[0])
bad_labels = np.full(bad_features.shape[0], -1)
neutral_labels = np.zeros(neutral_features.shape[0])

In [16]:
from sklearn.model_selection import train_test_split

X = np.vstack([good_features, bad_features, neutral_features])

y = np.hstack([good_labels, bad_labels, neutral_labels])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss = 'log_loss', penalty = 'l2', alpha = 1.75e-3, learning_rate='optimal', random_state = 42, max_iter = 4, tol = None, early_stopping=True, validation_fraction=0.1, n_iter_no_change=15,class_weight='balanced')

X_initial_fit = X_train[:int(X_train.shape[0] * 0.1)]
y_initial_fit = y_train[:int(y_train.shape[0] * 0.1)]

clf.fit(X_initial_fit, y_initial_fit)


SGDClassifier(alpha=0.00175, class_weight='balanced', early_stopping=True,
              loss='log_loss', max_iter=4, n_iter_no_change=15, random_state=42,
              tol=None)

In [18]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.6429920664903664


In [19]:
def partial_fit_step_train():
    for i in range(10):
        X_partial_fit = X_train[int(X_train.shape[0] * 0.05) * i:int(X_train.shape[0] * 0.1) * (i + 1)]
        y_partial_fit = y_train[int(y_train.shape[0] * 0.05) * i:int(y_train.shape[0] * 0.1) * (i + 1)]
    
        clf.partial_fit(X_partial_fit, y_partial_fit)
        y_pred = clf.predict(X_test)
    
        print(accuracy_score(y_test, y_pred))

In [20]:
partial_fit_step_train()

0.6467699282206272
0.6505477899508878
0.6800151114469211
0.6824707215715905
0.6945598791084246
0.6981488477521722
0.702304495655459
0.7058934642992066
0.7049489988666415
0.7072157159047979


In [21]:
def test():   
    y_pred = clf.predict(X_test)
    
    print(accuracy_score(y_test, y_pred))

In [22]:
test()

0.7072157159047979


In [23]:
with open ('../headlinesData/Good.csv') as good:
    good = good.read().split('\n')
    
with open ('../headlinesData/Bad.csv') as bad:
    bad = bad.read().split('\n')

with open ('../headlinesData/Neutral.csv') as neutral:
    neutral = neutral.read().split('\n')
    
csv_good = pd.DataFrame(good, columns=['headline'])
csv_bad = pd.DataFrame(bad, columns=['headline'])
csv_neutral = pd.DataFrame(neutral, columns=['headline'])

csv_good_good_vc = good_vc.transform(csv_good['headline'])
csv_bad_good_vc = good_vc.transform(csv_bad['headline'])
csv_neutral_good_vc = good_vc.transform(csv_neutral['headline'])

csv_good_v_t = v_t.transform(csv_good['headline'])
csv_bad_v_t = v_t.transform(csv_bad['headline'])
csv_neutral_v_t = v_t.transform(csv_neutral['headline'])

csv_good_sia = csv_good['headline'].apply(lambda x: sia.polarity_scores(x))
csv_bad_sia = csv_bad['headline'].apply(lambda x: sia.polarity_scores(x))
csv_neutral_sia = csv_neutral['headline'].apply(lambda x: sia.polarity_scores(x))

csv_good_features = np.hstack([csv_good_sia.apply(pd.Series).values, csv_good_good_vc.toarray(), csv_good_v_t.toarray()])

csv_bad_features = np.hstack([csv_bad_sia.apply(pd.Series).values, csv_bad_good_vc.toarray(), csv_bad_v_t.toarray()])

csv_neutral_features = np.hstack([csv_neutral_sia.apply(pd.Series).values, csv_neutral_good_vc.toarray(), csv_neutral_v_t.toarray()])

csv_good_labels = np.ones(csv_good_features.shape[0])

csv_bad_labels = np.full(csv_bad_features.shape[0], -1)

csv_neutral_labels = np.zeros(csv_neutral_features.shape[0])
        


In [24]:
X = np.vstack([good_features, bad_features, neutral_features])

y = np.hstack([good_labels, bad_labels, neutral_labels])

def test():  
    y_pred = clf.predict(X)
    
    print(accuracy_score(y, y_pred))

In [25]:
test()

0.7465805184009673


In [26]:
y_pred

array([-1.,  0.,  1., ..., -1.,  1., -1.])

In [27]:
import pickle as p

p.dump(clf, open('newsModel4.0.pkl', 'wb'))

p.dump(v_t, open('newsTFVectorizer4.0.pkl', 'wb'))

p.dump(good_vc, open('newsGoodVectorizer4.0.pkl', 'wb'))

p.dump(bad_vc, open('newsBadVectorizer4.0.pkl', 'wb'))

p.dump(neutral_vc, open('newsNeutralVectorizer4.0.pkl', 'wb'))

p.dump(sia, open('newsSIA4.0.pkl', 'wb'))